### Notes

Master DB has:
 - rep_test1.rep imported with default importer
 - gpx_1_0.gpx imported with platform NELSON created with sensor GPS
 
Slave DB has:
 - gpx_1_1.gpx imported with platform NELSON created with sensor GPS

In [1]:
from pepys_import.core.store.data_store import DataStore
from sqlalchemy.orm.session import make_transient
from sqlalchemy.orm import undefer

In [2]:
master_store = DataStore("", "", "", 0, db_name="master.sqlite", db_type="sqlite")
slave_store = DataStore("", "", "", 0, db_name="slave.sqlite", db_type="sqlite")

   @@@@ @@@@@        ______                      _                            _   
 @@@@@ @@..@@@@      | ___ \                    (_)                          | |  
  @@@     @@@@@@     | |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
  @@@     @@@@@@@@   |  __/ _ \ '_ \| | | / __| | | '_ ` _ \| '_ \ / _ \| '__| __|
 @@@@.     @@@@@@@   | | |  __/ |_) | |_| \__ \ | | | | | | | |_) | (_) | |  | |_ 
 @@ #@@ @   ...@@@@@ \_|  \___| .__/ \__, |___/ |_|_| |_| |_| .__/ \___/|_|   \__|
 @@   &        @@@@@            | |     __/ | ______          | |                   
Software Version :  0.0.10 


Database Type :  sqlite
Database Name :  master.sqlite
Database Host :  
-------------------------------------------------------------
   @@@@ @@@@@        ______                      _                            _   
 @@@@@ @@..@@@@      | ___ \                    (_)                          | |  
  @@@     @@@@@@     | |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
  @

In [3]:
with master_store.session_scope():
    res = master_store.session.query(master_store.db_classes.Sensor).all()
    print(res)
    print(len(res))

[Sensor(id=77f092, name=TA, host={shorten_uuid(self.host)}, host__name={self.host__name}, sensor_type={shorten_uuid(self.sensor_type_id)}, sensor_type__name={self.sensor_type_name}), Sensor(id=ed0e60, name=TA, host={shorten_uuid(self.host)}, host__name={self.host__name}, sensor_type={shorten_uuid(self.sensor_type_id)}, sensor_type__name={self.sensor_type_name}), Sensor(id=b572bb, name=SENSOR-1, host={shorten_uuid(self.host)}, host__name={self.host__name}, sensor_type={shorten_uuid(self.sensor_type_id)}, sensor_type__name={self.sensor_type_name}), Sensor(id=5a120e, name=SENSOR-1, host={shorten_uuid(self.host)}, host__name={self.host__name}, sensor_type={shorten_uuid(self.sensor_type_id)}, sensor_type__name={self.sensor_type_name}), Sensor(id=c425d2, name=GPS, host={shorten_uuid(self.host)}, host__name={self.host__name}, sensor_type={shorten_uuid(self.sensor_type_id)}, sensor_type__name={self.sensor_type_name})]
5


In [ ]:
with slave_store.session_scope():
    res = slave_store.session.query(slave_store.db_classes.SensorType).options(undefer("*")).all()
    print(res)
    print(len(res))
    
    slave_store.session.expunge_all()
    with master_store.session_scope():
        for r in res[0:1]:
            make_transient(r)
            master_store.session.add(r)
        print(master_store.session.new)
        print(r in master_store.session)
        master_store.session.flush()

In [ ]:
s = res[0]

In [ ]:
master_store.session.commit()

In [ ]:
# For each platform in slave db
#   Search platform details in master
#   If 0 matches, then just copy platform across
#   If 1 match, update slave db platform foreign key to be same as master platform foreign key
#   If 2 matches, fail

# Check that State objects have been updated